## **Initial Imports**

In [239]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample




Importing NLTK library and requirements for natural Language processing

In [240]:
import nltk 
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\anish\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anish\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## **Loading DATA**

In [241]:
df = pd.read_csv('spam.csv', encoding='latin-1')
df.sample(6)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
46,ham,Didn't you get hep b immunisation in nigeria.,NaN,NaN,NaN
3730,ham,Isn't frnd a necesity in life? imagine urself ...,NaN,NaN,NaN
710,ham,Height of Confidence: All the Aeronautics prof...,"this wont even start........ Datz confidence..""",NaN,NaN
4020,ham,We have to pick rayan macleran there.,NaN,NaN,NaN
3009,ham,Imagine Life WITHOUT ME... see.. How fast u ar...,NaN,NaN,NaN
1412,ham,"Wen ur lovable bcums angry wid u, dnt take it ...",NaN,NaN,NaN


## **Data Cleaning**


In [242]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB


In [243]:
# Dropping the columns with manynull values as illustrated by the info()

df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)

In [244]:
# Encoding the target variable from "ham" to 0 and "spam" to 1

encoder = LabelEncoder()
df['v1'] = encoder.fit_transform(df['v1'])

In [245]:
# checking for any duplicate values

df.duplicated().sum()

403

In [246]:
# Dropping the duplicates and checking the distribution of DATA

df = df.drop_duplicates(keep = 'first')
df['v1'].value_counts()

v1
0    4516
1     653
Name: count, dtype: int64

In [247]:
# Balancing the dataset as the number of ham messages is very large as compared to the number of spam messages

spam = df[df['v1'] == 1]
ham = df[df['v1'] == 0]

ham_downsampled = resample(ham,replace=False, n_samples = len(spam),random_state=42)

df = pd.concat([spam, ham_downsampled])

# Suffling
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df['v1'].value_counts()


v1
0    653
1    653
Name: count, dtype: int64

Now the data is balanced and cleaned for further processing

## **DATA Preprocessing**


In [248]:
def transform_text(text):
    text = text.lower()
    text = nltk.word_tokenize(text)


    # Lower-casing
    y = []
    for i in text:
        if i.isalnum():
            y.append(i)

    text = y[:]
    y.clear()


    # Removing special characters
    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)

    text = y[:]
    y.clear()


    # Stemming
    ps = PorterStemmer()
    
    for i in text:
        y.append(ps.stem(i))

    return " ".join(y)

In [249]:
# Transforming the message coulmn of the dataset

df["transformed_text"] = df["v2"].apply(transform_text)

In [250]:
df.sample(10)

,v1,v2,transformed_text
206,1,RT-KIng Pro Video Club>> Need help? info@ringt...,pro video club need help info call 08701237397...
550,1,Gr8 new service - live sex video chat on your ...,gr8 new servic live sex video chat mob see sex...
29,0,"swhrt how u dey,hope ur ok, tot about u 2day.l...",swhrt u dey hope ur ok tot u n care
563,1,"Bored of speed dating? Try SPEEDCHAT, txt SPEE...",bore speed date tri speedchat txt speedchat 80...
190,1,Free 1st week entry 2 TEXTPOD 4 a chance 2 win...,free 1st week entri 2 textpod 4 chanc 2 win 40...
1264,1,URGENT! Your Mobile No. was awarded å£2000 Bon...,urgent mobil award bonu caller prize final tri...
1025,0,"Jane babes not goin 2 wrk, feel ill after lst ...",jane babe goin 2 wrk feel ill lst nite fone al...
713,0,"They said if its gonna snow, it will start aro...",said gon na snow start around 8 9 pm tonit pre...
63,0,Yup ok thanx...,yup ok thanx
824,0,Haha... Yup hopefully we will lose a few kg b...,haha yup hope lose kg mon hip hop go orchard w...


## **Converting text into numeric features**

In [251]:
# Using Tfidf 

from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()

<!-- Naive bayes -->



In [252]:
x = tf.fit_transform(df['transformed_text']).toarray()
y = df['v1'].to_numpy()

## **Train Test Splitting**

In [253]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

## **Model Training**

In [254]:
# I am using Multinomial Naive-Bayes Model

from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

model = mnb.fit(x_train, y_train)

y_pred = model.predict(x_test)

## **Evaluation**

In [255]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [256]:
print("Accuracy of the model is : " , accuracy_score(y_test,y_pred))
print("Precision score of model is : ", precision_score(y_test,y_pred))


Accuracy of the model is :  0.9312977099236641
Precision score of model is :  0.9242424242424242


## **Checking the custom inputs**

In [257]:
def check(s):
    x = transform_text(s)
    vec = tf.transform([s])
    return model.predict(vec)

In [260]:
Example = input("Enter the SMS: ")

if check(Example):
    print("Spam")
else:
    print("Not Spam")

Spam
